In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
cleint_prof_with_cbil_label = dataiku.Dataset("cleint_prof_with_cbil_label")
cleint_prof_with_cbil_label_df = cleint_prof_with_cbil_label.get_dataframe()

In [0]:
cleint_prof_with_cbil_label_df.fraudulent_account.value_counts()

In [0]:
df = cleint_prof_with_cbil_label_df

In [0]:
df.columns

In [0]:
df[df['fraudulent_account'] ==1].credit_score.describe()

In [0]:
df[df['fraudulent_account'] ==0].credit_score.describe()

In [0]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Function to generate new values based on fraud label
# Function to generate integer-based values
def generate_clustered_values(row):
    if row['fraudulent_account'] == 0:
        credit_score = int(np.clip(np.random.normal(loc=700, scale=60), 300, 850))
        utilization = int(np.clip(np.random.normal(loc=30, scale=15), 0, 100))  # as percentage
    else:
        credit_score = int(np.clip(np.random.normal(loc=520, scale=70), 300, 850))
        utilization = int(np.clip(np.random.normal(loc=75, scale=15), 0, 100))  # as percentage

    return pd.Series([credit_score, utilization])

In [0]:
# Apply to dataframe
df[['credit_score_new', 'current_utilization_new']] = df.apply(generate_clustered_values, axis=1)

In [0]:
import random

# Define suspicious domains
fishy_domains = [
    'secure-mail.cc', 'xmailz.co', 'freemail.pw', 'tempbox.site', 'phishy.link',
    'mail-temp.com', 'shadyinbox.net', 'noreplynow.co', 'inboxproxy.cc',
    'getnada.com', 'mailinator.com', '10minutemail.net', 'tempmail.io',
    'fakeinbox.com', 'disposablemail.org', 'trashmail.cc', 'anonaddy.me',
    'sneaky.email', 'quickmail.bid', 'instamail.host', 'mymail.ink',
    'guerrillamail.com', 'deadmail.online', 'click2mail.pw', 'nowhereinbox.info',
    'hackmail.xyz', 'darkmailer.zone', 'burnmail.org'
]
# Get indices of fraudulent rows
fraud_indices = df[df['fraudulent_account'] == 1].index

# Sample 80% of fraudulent indices to modify
num_to_modify = int(0.8 * len(fraud_indices))
indices_to_modify = np.random.choice(fraud_indices, size=num_to_modify, replace=False)

# Modify the email addresses
for idx in indices_to_modify:
    local_part = df.loc[idx, 'email'].split('@')[0]
    new_domain = random.choice(fishy_domains)
    df.at[idx, 'email'] = f"{local_part}@{new_domain}"

In [0]:
df

In [0]:
today = pd.to_datetime("today")

df['dob'] = pd.to_datetime(df['dob'])
# Compute age
df['age'] = df['dob'].apply(lambda dob: today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day)))

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

modifying_truth_of_credit_score_df = df # For this sample code, simply copy input to output


# Write recipe outputs
modifying_truth_of_credit_score = dataiku.Dataset("modifying_truth_of_credit_score")
modifying_truth_of_credit_score.write_with_schema(modifying_truth_of_credit_score_df)